In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/ruyogagp/medical_interpretability')
import numpy as np
from pysurvival.models import BaseModel
from pysurvival import utils
import scipy
import pandas as pandas
import copy
import random
from sklearn.preprocessing import StandardScaler
from source.utils import create_correlated_var
from pysurvival.models.simulations import SimulationModel
from lifelines import CoxPHFitter
import pandas as pd

In [3]:
import networkx as nx
from cga import cga
from itertools import cycle
import tqdm as tqdm
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split

## Helper Functions

In [37]:
def fit_coxph(df):
    cph = CoxPHFitter()
    cph.fit(df, 'time', 'event')
    cph.print_summary()

def fit_coxph_norm(df):
    standard_scaler = StandardScaler()
    for col in df.columns:
        if col == 'time' or col == 'event':
            pass
        df[col] = standard_scaler.fit_transform(df[[col]])
    cph = CoxPHFitter()
    cph.fit(df, 'time', 'event')
    cph.print_summary()


def save_orig(df, name, output_dir):
    train, valid = train_test_split(df, test_size=0.3)
    train.to_csv(
        f"{output_dir}/{name}_train.csv",
        index=False,
    )
    valid.to_csv(
        f"{output_dir}/{name}_valid.csv",
        index=False,
    )
    print(f"Saved {output_dir}/{name}_train.csv")
    print(f"Saved {output_dir}/{name}_valid.csv")

def df2csv(
        df: pd.DataFrame,
        name: str,
        output_dir: str,
):
    """
    Writes csv given a dataframe + name
    """
    train, valid = train_test_split(df, test_size=0.3)
    train.to_csv(
        f"{output_dir}/{name}_train_details.csv",
        index=False,
    )
    valid.to_csv(
        f"{output_dir}/{name}_valid_details.csv",
        index=False,
    )

    train_df = train.loc[:, ['x_orig', 'y_orig', 'time_orig', 'event_orig']]
    valid_df = valid.loc[:, ['x_orig', 'y_orig', 'time_orig', 'event_orig']]
    train_df.rename(columns=dict(x_orig='x',
                                 y_orig='y',
                                 time_orig='time',
                                 event_orig='event'), inplace=True)

    valid_df.rename(columns=dict(x_orig='x',
                                 y_orig='y',
                                 time_orig='time',
                                 event_orig='event'), inplace=True)
    train_df.to_csv(
        f"{output_dir}/{name}_train.csv",
        index=False,
    )
    valid_df.to_csv(
        f"{output_dir}/{name}_valid.csv",
        index=False,
    )

## Simulation Model with correlations

In [38]:
class SimulationModelWithCorrelations(SimulationModel):
    """
    Subclasses `SimulationModel` to generated data from an predefined
    risk factor.
    """

    def generate_data(self,
                      df: pd.DataFrame,
                      feature_weights: list,
                      feature_names: list,
                      ):

        def risk_function(x_std, feature_weights):
            """ Calculating the risk function based on the given risk type """

            # Dot product
            risk = np.dot(x_std, feature_weights )

            # Choosing the type of risk
            if self.risk_type.lower() == 'linear' :
                return risk.reshape(-1, 1)

            elif self.risk_type.lower() == 'square' :
                risk = np.square(risk*self.risk_parameter)


            elif self.risk_type.lower() == 'gaussian' :
                risk = np.square(risk)
                risk = np.exp( - risk*self.risk_parameter)

            return risk.reshape(-1, 1)

        input_data = df.loc[:, feature_names].to_numpy()
        self.dataset = copy.deepcopy(df)
        num_samples = input_data.shape[0]
        X_std = self.scaler.fit_transform(input_data)
        BX = risk_function(X_std, feature_weights)

        # Building the survival times
        T = self.time_function(BX)
        C = np.random.normal(loc=self.censored_parameter, scale=5, size=num_samples)
        C = np.maximum(C, 0.0)
        time = np.minimum(T, C)
        E = 1.0 * (T == time)

        # Building dataset
        self.dataset = copy.deepcopy(df)
        self.dataset['time'] = time
        self.dataset['event'] = E

        # Building the time axis and time buckets
        self.times = np.linspace(0.0, max(self.dataset["time"]), self.bins)
        self.get_time_buckets()

        # Building baseline functions
        self.baseline_hazard = self.hazard_function(self.times, 0)
        self.baseline_survival = self.survival_function(self.times, 0)

        # Printing summary message
        message_to_print = "Number of data-points: {} - Number of events: {}"
        print(message_to_print.format(num_samples, sum(E)))
        return self.dataset

## Correlation Case Graph

In [39]:
@cga.node
def correlate(x: float, rnorm_vector:float, noise:float) -> float:
    """
    :param x: exisiting data to correlate
    :param coeff: correlation coefficient
    :param noise: noise variable
    :return: variable correlated by coeff to the exisiting variable x
    """
    correlate = create_correlated_var(x, rnorm_vector,
                                      mu=np.mean(x),
                                      sd=np.std(x),
                                      empirical=True,
                                      r=-0.75)
    return correlate + noise

@cga.node
def sample_random_normal(noise:float)->float:
    """
    :param n: sample size
    :param noise: noise variable
    :return: random normal variable
    """
    return np.random.normal(size=100) + noise

@cga.node
def correlation_coefficient(coeff:float) -> float:
    return coeff

class CorrelationCaseGraph(cga.Graph):
    def __init__(self):
        """
        causal graph for correlation case
        :param n: number of data points
        :param coeff: desired correlation coefficient between the two variables
        """
        noise = cga.node(lambda: np.random.normal(scale=0.0001, size=100))
        rnorm_vector = cga.node(lambda: np.random.normal(size=100))
        self.rnorm = rnorm_vector(name="rnorm")
        self.noise_x = noise(name="noise_x")
        self.noise_y = noise(name="noise_y")
        self.x = sample_random_normal(self.noise_x, name='x')
        self.y = correlate(self.x, self.rnorm, self.noise_y, name='y')
        super().__init__([self.x, self.y])

    def get_interventions(self,
                          sim: SimulationModelWithCorrelations,
                          n_iterations: int,
                          feature_weights: list,
                          ) -> pd.DataFrame:
        data = None
        for node in [self.noise_x, self.noise_y]:
            for _ in tqdm.trange(n_iterations, desc=f"Intervention {node.name}"):
                # resample noise
                orig, intervention = self.sample_do(action=cga.Resample(node))
                row = {'modified_attribute': [node.name] * 100}
                # add orig + do to the dictionary
                row.update({
                    n.name + "_orig": v
                    for n, v in orig.items()
                })
                row.update({
                    n.name + "_do": v
                    for n, v in intervention.items()
                })

                data = row if data is None else data
                for key in row.keys():
                    row[key] = row[key].tolist() if isinstance(row[key], np.ndarray) else row[key]
                    data[key].extend(row[key])
        df = pd.DataFrame(data)

        orig_cols = ['x_orig', 'y_orig']
        orig_df = sim.generate_data(df, feature_names=orig_cols,
                                    feature_weights=feature_weights)

        xdf = orig_df.loc[orig_df.modified_attribute=='noise_x']\
            .loc[:, ['x_orig', 'y_orig', 'time', 'event']]\
            .rename(columns=dict(x_orig='x', y_orig='y'))
        ydf = orig_df.loc[orig_df.modified_attribute=='noise_y']\
            .loc[:, ['x_orig', 'y_orig', 'time', 'event']]\
            .rename(columns=dict(x_orig='x', y_orig='y'))

        df['event_orig'] = orig_df.event
        df['time_orig'] = orig_df.time

        do_cols = ['x_do', 'y_do']
        do_df = sim.generate_data(df, feature_names=do_cols,
                                  feature_weights=feature_weights)
        df['event_do'] = do_df.event
        df['time_do'] = do_df.time

        return xdf, ydf, df

    def test_intervention(self, n_iterations):
        for node in [self.noise_x, self.noise_y]:
            for _ in tqdm.trange(n_iterations, desc=f"Intervention {node.name}"):
                # resample noise
                orig, intervention0, intervention1, intervention2 = self.sample_do(action=cga.Resample(node))
        return orig, intervention0, intervention1, intervention2

## Sample from Graph

In [40]:
# Sample features
data = None
correlation_graph = CorrelationCaseGraph()
for _ in tqdm.trange(100, desc='sampling'):
    result = correlation_graph.sample()
    data = result if data is None else data
    for key in result.keys():
        result[key] = result[key].tolist() if isinstance(result[key], np.ndarray) else result[key]
        data[key].extend(result[key])
del data[correlation_graph.noise_x]
del data[correlation_graph.noise_y]
del data[correlation_graph.rnorm]

# Generate data
training_features = pd.DataFrame(data)
sim = SimulationModelWithCorrelations(risk_type='linear', alpha=1.0, beta=5.0, censored_parameter=5.0, survival_distribution='weibull')
feature_weights = [np.log(2), np.log(1.5)]
feature_names = [correlation_graph.x, correlation_graph.y]
training_df = sim.generate_data(training_features, feature_weights=feature_weights, feature_names=feature_names)

# Check correlations
training_df.corr()

sampling: 100%|██████████| 100/100 [00:00<00:00, 623.20it/s]


Number of data-points: 10100 - Number of events: 7968.0


,x,y,time,event
x,1.000000,-0.733541,-0.138720,0.010682
y,-0.733541,1.000000,0.051837,0.008678
time,-0.138720,0.051837,1.000000,0.812486
event,0.010682,0.008678,0.812486,1.000000


## Save Training Data

In [41]:
directory = '/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/correlation'
save_orig(training_df, name='p-0.75', output_dir=directory)

Saved /data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/correlation/p-0.75_train.csv
Saved /data/analysis/ag-reils/ag-reils-shared/cardioRS/data/interpretability/correlation/p-0.75_valid.csv


## Resample Features

In [42]:
# Resample from graph
correlation_graph = CorrelationCaseGraph()
feature_weights = [np.log(2), np.log(1.5)]
sim = SimulationModelWithCorrelations(risk_type='linear', alpha=1.0, beta=5.0, censored_parameter=5.0, survival_distribution='weibull')
xdf, ydf, resampling_df = correlation_graph.get_interventions(sim=sim, n_iterations=50, feature_weights=feature_weights)

# Check correlations
xdf.corr()

Intervention noise_y: 100%|██████████| 50/50 [00:00<00:00, 332.34it/s]


Number of data-points: 10100 - Number of events: 8016.0
Number of data-points: 10100 - Number of events: 8057.0


,x,y,time,event
x,1.000000,-0.730646,-0.148846,0.012297
y,-0.730646,1.000000,0.046590,-0.010465
time,-0.148846,0.046590,1.000000,0.808448
event,0.012297,-0.010465,0.808448,1.000000


## Save Attribution Data

In [43]:
xdf.to_csv(f'{directory}/p-0.75_attribute_x.csv', index=False)
ydf.to_csv(f'{directory}/p-0.75_attribute_y.csv', index=False)
resampling_df.to_csv(f'{directory}/p-0.75_attribute_details.csv', index=False)